# Квантование с Pytorch


In [1]:
#!pip3 install torch==1.5.0 torchvision==1.6.0

import os
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader

import torch.quantization
from torch.quantization import QuantStub, DeQuantStub

import time

Загрузим данные MNIST для обучения и тестирования

In [2]:
args={}
args['batch_size']=500
args['test_batch_size']=500
args['epochs']=5  #The number of Epochs is the number of times you go through the full dataset.
args['lr']=0.005 #Learning rate is how fast it will decend.
args['seed']=1 #random seed
args['log_interval']=20
args['cuda']=True

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=args['test_batch_size'],
                                          shuffle=True, num_workers=16, pin_memory=True)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=args['test_batch_size'],
                                         shuffle=False, num_workers=16, pin_memory=True)

Определите некоторые вспомогательные функции и классы, которые помогут нам отслеживать статистику и точность данных обучения/тестирования.

In [4]:
def print_size_of_model(model):
    """ Prints the real size of the model """
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

def load_model(quantized_model, model):
    """ Loads in the weights into an object meant for quantization """
    state_dict = model.state_dict()
    model = model.to('cpu')
    quantized_model.load_state_dict(state_dict)

In [5]:
def train(model, epoch, loss_fn, optimizer, train_loader):
    model.train()
    if  args['cuda']:
        model.cuda()
    else:
        model.to('cpu')
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()

        optimizer.zero_grad()
        output = model(data)

        loss = loss_fn(output, target)

        loss.backward()

        optimizer.step()

        #Print out the loss periodically.
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, test_loader):
    model.eval()
    import time

    start = time.time()
    if  args['cuda']:
        model.cuda()
    else:
        model.to('cpu')

    correct = 0
    for data, target in test_loader:
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        #data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()



    end = time.time()
    acc = 100. * correct / len(test_loader.dataset)
    print('\nTest set:  Accuracy: {}/{} ({:.0f}%)  Time: {}\n'.format(correct, len(test_loader.dataset),acc, end - start))

    return acc


def train_and_eval(model, train_loader, test_loader):
    if args['cuda']:
        model.cuda()

    history = []
    loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
    optimizer = optim.Adam(model.parameters(), lr=args['lr'])
    for epoch in range(1, args['epochs'] + 1):

        train(model, epoch, loss_fn, optimizer, train_loader)
        acc = test(model, test_loader)
        history.append(acc)

    return acc

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, q=False):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, stride=1, bias=False)
        self.conv2 = nn.Conv2d(8, 8, 3, stride=5, bias=False)
        self.fc1 = nn.Linear(32, 1000)
        self.fc2 = nn.Linear(1000, 10)

        self.q = q
        if self.q:
            self.quant = QuantStub()
            self.dequant = DeQuantStub()


    def forward(self, x):

        if self.q:
          x = self.quant(x)

        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        F.relu(x)
        x = self.fc1(x)
        output = self.fc2(x)

        if self.q:
          output = self.dequant(output)
        return output

Определим простую CNN, которая классифицирует изображения MNIST.

In [7]:
model = SimpleCNN(q=False)
print_size_of_model(model)

Size (MB): 0.176511


In [8]:
hist = train_and_eval(model, train_loader, test_loader)

/usr/local/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304790
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.114714
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.693313
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.695130
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.637412
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.654544

Test set:  Accuracy: 8305/10000 (83%)  Time: 0.866278886795044

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.588512
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.531931
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.555876
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.479359
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.439697
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.481069

Test set:  Accuracy: 8532/10000 (85%)  Time: 0.7021305561065674

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.488484
Train Epoch: 3 [10000/60000 (17%)]	Loss: 0.411793
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.493540
Train Epoch: 3 [30000/60000 (50%)]	Loss: 0.502778
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.524337
Train Epoch: 3 [50000/60000 (83%)]

### Post-training quantization

Определим новую архитектуру квантовой сети, в которой мы также определим заглушки квантования и деквантования, которые будут важны в начале и в конце.


In [20]:
qmodel = SimpleCNN(q=True)

load_model(qmodel, model)

test(qmodel, test_loader)


Test set:  Accuracy: 8779/10000 (88%)  Time: 0.8252761363983154



tensor(87.7900)

In [21]:
print_size_of_model(qmodel)

Size (MB): 0.176575


Статическое квантование после обучения включает в себя не только преобразование весов из числа с плавающей запятой в целое число, как при динамическом квантовании, но и выполнение дополнительных
этап первой подачи пакетов данных через сеть и вычисления результирующих распределений различных активаций (в частности,
это делается путем вставки модулей наблюдателей в разные
точки, записывающие эти данные). Эти распределения затем используются для определения того, как конкретно следует квантовать различные активации.
время вывода (простым методом было бы просто разделить весь диапазон активаций на 256 уровней.
Важно отметить, что этот дополнительный шаг позволяет нам передавать квантованные значения между операциями вместо преобразования этих значений в числа с плавающей запятой, а затем обратно в целые числа между каждой операцией.
что приводит к значительному ускорению.

In [22]:
qmodel.qconfig = torch.quantization.default_qconfig
print(qmodel.qconfig)

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, quant_min=0, quant_max=127){}, weight=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric){})


In [ ]:
# qmodel.to('cpu')
args['cuda']=False

qmodel.qconfig = torch.quantization.default_qconfig
print(qmodel.qconfig)

torch.quantization.prepare(qmodel, inplace=True)
test(qmodel, train_loader)
print('Post Training Quantization Prepare: Inserting Observers')
print('\n Conv1: After observer insertion \n\n', qmodel.conv1)

print('#'*20)


print('Post Training Quantization: Calibration done')
torch.quantization.convert(qmodel, inplace=True)
print('Post Training Quantization: Convert done')
print('\n Conv1: After fusion and quantization \n\n', qmodel.conv1)
print("Size of model after quantization")
print_size_of_model(qmodel)

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, quant_min=0, quant_max=127){}, weight=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric){})



Test set:  Accuracy: 52011/60000 (87%)  Time: 7.279160022735596

Post Training Quantization Prepare: Inserting Observers

 Conv1: After observer insertion 

 Conv2d(
  1, 8, kernel_size=(3, 3), stride=(1, 1), bias=False
  (activation_post_process): MinMaxObserver(min_val=-1.812164068222046, max_val=1.812164068222046)
)
####################

Test set:  Accuracy: 52011/60000 (87%)  Time: 7.016843557357788


Test set:  Accuracy: 52011/60000 (87%)  Time: 7.141284704208374

Post Training Quantization: Calibration done
Post Training Quantization: Convert done

 Conv1: After fusion and quantization 

 QuantizedConv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), scale=0.028538016602396965, zero_point=64, bias=False)
Size of model after quantization
Size (MB): 0.051583


In [24]:
0.051583/0.176575

0.2921308225966303

In [25]:
args['cuda']=False

a = test(qmodel, test_loader)


Test set:  Accuracy: 8741/10000 (87%)  Time: 0.9581553936004639



In [ ]:
print_size_of_model(qmodel)

Size (MB): 0.051583


In [19]:
print_size_of_model(model)

Size (MB): 0.176511
